In [24]:
#!/usr/bin/env python
"""
CEVAE moargs.latent_dimensionel on IHargs.latent_dimensionP
"""
##################### Neeargs.latent_dimension TF1
# import edward as ed
# import tensorflow as tf
# from utils import fc_net, get_y0_y1
# from edward.models import Bernoulli, Normal
# from progressbar import ETA, Bar, Percentage, ProgressBar
###############################################################
import os
# import json
# import time
# import math
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.stats import sem
import tensorflow.keras as tfk
import tensorflow_probability as tfp
import tensorflow.keras.layers as tfkl
tfd,tfpl = tfp.distributions,tfp.layers
import tensorflow.keras.backend as tfkb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from datasets import IHDP
from evaluation import Evaluator
from argparse import ArgumentParser

parser = ArgumentParser()
################### CEVAE
parser.add_argument('-reps', type=int, default=10)
parser.add_argument('-earl', type=int, default=10)
parser.add_argument('-lr', type=float, default=0.001)
parser.add_argument('-decay', type=float, default=0.001)
parser.add_argument('-opt', choices=['adam', 'adamax'], default='adam')
parser.add_argument('-epochs', type=int, default=100)     
parser.add_argument('-print_every', type=int, default=10)
################### BASIC
parser.add_argument('--num_parallel_reads', type = int, default = 10, help = 'random seed')
parser.add_argument('--random_seed', type = int,default = 9418, help = 'random seed')
parser.add_argument('--mode', type = str, default = 'train', help = 'train or evaluate')
args = parser.parse_args([])
args.true_post = True
#################################From CEVAE
dataset = IHDP(replications=args.reps)
dimx = 25
scores = np.zeros((args.reps, 3))
scores_test = np.zeros((args.reps, 3))
#################################IHDP Data
# data information 
t_bin_dim = 1
y_dim, default_y_scale = 1,tf.exp(0.)
M = None        # batch size during training
z_dim = 20          # latent z dimension
lamba = 1e-4    # weight decay
nh, h = 3, 200  # number and size of hidden layers
binfeats = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
numfeats = [i for i in range(25) if i not in binfeats]
x_bin_dim = len(binfeats)
x_num_dim = len(numfeats)
################################# some functions needed
def findfile(start, name):
    ans = []
    for relpath, dirs, files in os.walk(start):
        for file in files:
            if name in file:
                full_path = os.path.join(start, file)
                ans.append(full_path)
                # print(os.path.normpath(os.path.abspath(full_path)))
    return ans

In [47]:
# create datasets
from sklearn.preprocessing import StandardScaler
# get data
data_all = []
for data in findfile("./datasets/IHDP/csv",'.csv'):
    try:
        data_all = data_all.append(pd.read_csv(data, header = None), ignore_index = True)
    except:
        data_all = pd.read_csv(data, header = None)
data_all.columns = ['treatment', 'y_factual', 'y_cfactual', 'mu0', 'mu1', ] + ['x'+str(i) for i in range(25)]
# process
y_mean,y_std = np.mean(data_all['y_factual']),np.std(data_all['y_factual'])
data_all['y_factual_scale'] = (data_all['y_factual'] - y_mean) / y_std
data_all['x13'] = data_all['x13'] - 1
# set random seeds:
np.random.seed(7)
data_train, data_valid = train_test_split(data_all, test_size=0.2, )
# data_train.to_csv("./datasets/IHDP/csv_create/train.csv", header = None, index =None)
# data_valid.to_csv("./datasets/IHDP/csv_create/valid.csv", header = None, index =None)
data_all

,treatment,y_factual,y_cfactual,mu0,mu1,x0,x1,x2,x3,x4,...,x16,x17,x18,x19,x20,x21,x22,x23,x24,y_factual_scale
0,1,11.267344,6.314020,6.151308,11.160778,-0.528603,-0.343455,1.128554,0.161703,-0.316603,...,1,1,1,0,0,0,0,0,0,0.083537
1,0,3.966599,9.136500,2.633272,10.312341,-1.736945,-1.802002,0.383828,2.244320,-0.629189,...,1,1,1,0,0,0,0,0,0,-0.426812
2,0,3.403815,10.527021,4.532332,10.855350,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,...,0,1,1,0,0,0,0,0,0,-0.466153
3,0,3.471264,10.890649,4.310770,10.805230,0.390083,0.596582,-1.850350,-0.879606,-0.004017,...,0,1,1,0,0,0,0,0,0,-0.461438
4,0,4.531366,11.504950,5.621538,11.070719,-1.045229,-0.602710,0.011465,0.161703,0.683672,...,1,1,1,0,0,0,0,0,0,-0.387333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7465,0,3.633425,9.458109,3.398584,8.162824,-0.007654,-0.202946,-0.360898,0.161703,-0.316603,...,0,1,0,0,0,0,0,0,0,-0.450102
7466,1,7.692058,5.441664,7.195693,8.912948,0.727295,-0.202946,-0.733261,-0.879606,0.808706,...,1,1,0,0,0,0,0,0,0,-0.166389
7467,0,1.785687,9.759147,3.485027,8.187941,1.181234,0.196818,-1.477987,0.161703,0.746189,...,1,1,0,0,0,0,0,0,0,-0.579266
7468,0,8.585642,8.943839,8.154517,9.038037,-0.288664,-0.202946,-1.477987,-0.879606,1.621430,...,1,1,0,0,0,0,0,0,0,-0.103924


In [49]:
def ihdp_fn(data):
    data = tf.strings.to_number(tf.strings.split(data, sep=','))
    t, y, y_cf = data[0:1], data[1:2], data[2:3]
    mu_0, mu_1, x = data[3:4], data[4:5], data[5:30]
    x_bin = x[..., x_num_dim:]  # binary inputs
    x_num = x[..., 0:x_num_dim]  # continuous inputs
    return x,tf.concat([t,y],-1)
    
def input_fn_csv(path,name, batch_size=64, shuffle_size=0):
    all_files = findfile(path,name)
    ds = tf.data.TextLineDataset(
        all_files,
        compression_type= None,
        num_parallel_reads=int(args.num_parallel_reads),
        buffer_size=64*1024*1024# 可选参数，多线程并行读多个文件
    ).repeat(args.reps)
    if shuffle_size > 0:
        ds = ds.shuffle(shuffle_size)
    ds = ds.map(ihdp_fn).batch(batch_size)
    return ds

loss_dict = {'huber': tf.keras.losses.Huber(delta=50),
             'bce' : tf.keras.losses.BinaryCrossentropy(from_logits=False),
             'mae' : tf.keras.losses.MeanAbsoluteError(),
             'mse' : tf.keras.losses.MeanSquaredError(),
             'msle' : tf.keras.losses.MeanSquaredLogarithmicError(),
             'logcosh' : tf.keras.losses.LogCosh(reduction="auto", name="log_cosh"),
            }

train_dataset = input_fn_csv("./datasets/IHDP/csv_create",'train')
valid_dataset = input_fn_csv("./datasets/IHDP/csv_create",'valid')

for data in input_fn_csv("./datasets/IHDP/csv_create",'train',10):
    break
data

(<tf.Tensor: shape=(10, 25), dtype=float32, numpy=
 array([[ 1.1596175 ,  0.5965822 , -0.360898  ,  0.16170253,  0.6836721 ,
         -0.8577868 ,  1.        ,  0.        ,  1.        ,  1.        ,
          0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          1.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
        [ 1.224466  ,  0.19681813, -1.4779869 , -0.879606  ,  0.18353444,
          2.1232939 ,  0.        ,  0.        ,  1.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
          1.        ,  1.        ,  1.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
        [-1.7369449 , -2.2017663 ,  1.8732799 ,  2.2443194 , -2.2546365 ,
          0.79836917,  0.        ,  0.        ,  0.        ,  1.        ,
          0.        ,  0.        ,  1.        ,  0.        

In [45]:
class tarnet(tf.keras.Model):
    def __init__(self):
        super(tarnet, self).__init__()
        ########################################
        # networks
        self.activation = 'elu'
        # CEVAE Model (decoder)
        self.share_bottom = tfk.Sequential(
            [
                tfkl.InputLayer([x_bin_dim+x_num_dim]),
                tfkl.Dense(200,activation = self.activation),
                tfkl.Dense(200,activation = self.activation),
                tfkl.Dense(200,activation = self.activation),
            ])
        self.tower_t0 = tfk.Sequential(
            [
                tfkl.InputLayer([200]),
                tfkl.Dense(100,activation = self.activation),
                tfkl.Dense(100,activation = self.activation),
                tfkl.Dense(1,activation = None),
            ])

        self.tower_t1 = tfk.Sequential(
            [
                tfkl.InputLayer([200]),
                tfkl.Dense(100,activation = self.activation),
                tfkl.Dense(100,activation = self.activation),
                tfkl.Dense(1,activation = None),
            ])
    def call(self, data, training=False, serving=False):
        # Dataset_inp
        hidden_bottom = self.share_bottom(data)
        y_t0 = self.tower_t0(hidden_bottom)
        y_t1 = self.tower_t0(hidden_bottom)
    
        output = tf.concat([y_t0,y_t1],-1)
        return output

def tarnet_loss(data,pred):
    # get preds
    y_t0,y_t1 = pred[...,0],pred[...,1]
    # read labels
    t_real,y_real = data[...,0], data[...,1]
    # calculate loss
    loss = tf.math.square(y_t0-y_real) * (1-t_real) + tf.math.square(y_t1-y_real) * (t_real)
    loss_mean = tfkb.sum(loss)

    return loss_mean

from tensorflow.keras.optimizers import SGD
def build_model():
    model = tarnet()
    sgd_lr = 1e-5
    momentum = 0.9
    model.compile(
        optimizer=SGD(
            lr=sgd_lr, momentum=momentum, nesterov=True),
        loss = tarnet_loss
        )
    return model

model = build_model()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="fit_logs/", histogram_freq=1)
model.fit(
    train_dataset,
    # validation_data = valid_dataset, 
    callbacks=[tensorboard_callback],
    epochs = 10,
    )

/usr/local/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/10
934/934 [==============================] - 5s 4ms/step - loss: 58.4754
Epoch 2/10
934/934 [==============================] - 4s 5ms/step - loss: 58.0324
Epoch 3/10
934/934 [==============================] - 4s 4ms/step - loss: 57.8874
Epoch 4/10
934/934 [==============================] - 4s 5ms/step - loss: 57.7924
Epoch 5/10
934/934 [==============================] - 5s 5ms/step - loss: 57.7210
Epoch 6/10
934/934 [==============================] - 4s 5ms/step - loss: 57.6630
Epoch 7/10
934/934 [==============================] - 5s 5ms/step - loss: 57.6131
Epoch 8/10
934/934 [==============================] - 5s 5ms/step - loss: 57.5689
Epoch 9/10
934/934 [==============================] - 5s 6ms/step - loss: 57.5291
Epoch 10/10
934/934 [==============================] - 6s 6ms/step - loss: 57.4926
